<a href="https://colab.research.google.com/github/sohomghosh/FinSim_Financial_Hypernym_detection/blob/main/STS_FIN_HYP_final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.7 MB/s 
     |████████████████████████████████| 3.8 MB 10.9 MB/s 
     |████████████████████████████████| 1.2 MB 34.7 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 6.5 MB 33.8 MB/s 
     |████████████████████████████████| 895 kB 39.4 MB/s 
     |████████████████████████████████| 596 kB 33.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=bfdd3c56bd9ef85198e9ad4ec3f4e41cd6d008f9826cda4f57e4ccdbe4dcc71e
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
import json
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer, losses, SentencesDataset, evaluation, InputExample, util
from torch.utils.data import DataLoader
import torch

# Evaluation Metrics

In [ ]:
def compute_average_rank(Y_test, predictions):
  ranks = []
  for idx, y in enumerate(Y_test):
    ranks.append(get_rank(Y_test[idx], predictions[idx]))
  return np.mean(ranks)

def get_rank(gold, list_predictions, max_k=3):
  list_predictions = list_predictions[:max_k]
  try:
    rank = list_predictions.index(gold) + 1
  except ValueError:
    rank = max_k + 1
  return rank

# Reading Augmented Datasets

NOTE: The original dataset has been provided by the organizers https://sites.google.com/nlg.csie.ntu.edu.tw/finnlp2021/shared-task-finsim

This dataset has to be augmented with external data from <br>
* Investopedia https://www.investopedia.com/financial-term-dictionary-4769738 <br>
* FIBO https://spec.edmcouncil.org/fibo/OWL <br>
* DBPedia https://lookup.dbpedia.org/api/search/?query=term (Replace term with each of the financial terms) <br>

Details regarding the acronym detector module within Spacy is available at https://github.com/allenai/scispacy

Due to copyright conatraints, we cannot release the augmented dataset.

In [ ]:
my_dict = {"Bonds":0, "Forward":1, "Funds":2, "Future":3, "MMIs":4, "Option":5, "Stocks":6, "Swap":7, "Equity Index":8, "Credit Index":9, "Regulatory Agency":10, "Central Securities Depository":11, "Stock Corporation":12, "Credit Events":13, "Debt pricing and yields":14, "Parametric schedules":15, "Securities restrictions":16}

data_aug_train = pd.read_csv("data_aug_train.csv")
data_aug_valid = pd.read_csv("data_aug_valid.csv") 

#  Each of the above files has 5 columns 'term', 'term_definition', 'label', 'label_definition' and 'label_text'. 
# 'term' and 'label' have been provided by the organizers
# 'term_definition' and 'label_definition' have been obtained through the data augmentation steps
# 'label_text' contains numeric mappings of 'labels' using the dictionary my_dict

label_df = pd.read_csv("labels_definition.csv") # It has 17 rows i.e. one for each label. It has 2 columns 'label' and 'label_definition'. 'label_definition' is the expanded form of 'label' and has been obtained fro FIBO & internet. 'source' column i.e. the source from where the definition is collected is optional
if 'source' in label_df.columns:
  del label_df['source']
label_df_new = label_df.copy()
label_df_new['label_text'] = label_df['label'].apply(lambda x : my_dict[x])
label_df_new = label_df_new.sort_values('label_text').copy()


# Storing hierarchy of labels

In [ ]:
level0_dict = {"Bonds":"SEC", "MMIs":"SEC", "Stocks":"SEC", "Securities restrictions":"SEC", "Parametric schedules":"SEC",
          "Forward":"DER", "Option":"DER", "Swap": "DER",
          "Future" :"FBC", "Credit Events":"FBC", "Central Securities Depository":"FBC", "Regulatory Agency":"FBC",
         "Equity Index":"IND","Credit Index":"IND",
         "Funds":"CIV",
        "Debt pricing and yields":"MD",
        "Stock Corporation":"BE"}

level1_dict= {"Bonds":"Debt", "MMIs":"Debt", "Stocks":"Equities", "Securities restrictions":"Securities", "Parametric schedules":"Securities",
          "Forward":"Derivatives Contracts", "Option":"Derivatives Contracts", "Swap": "Derivatives Contracts",
          "Future" :"Financial Instruments", "Credit Events":"Debts and equities", "Central Securities Depository":"Functional Entities", "Regulatory Agency":"Functional Entities",
         "Equity Index":"Market Indices",
        "Credit Index":"Market Indices",
         "Funds":"Funds",
        "Debt pricing and yields":"Debt Temporal",
        "Stock Corporation":"Legal Entities"}

# Algorithm to generate negative samples from existing training set

In [ ]:
k = 0.4
sent1_li, sent2_li, label_li = [], [], []
for train_sent, label_sent, label, term in zip(data_aug_train['term_definition'].values, data_aug_train['label_definition'].values, data_aug_train['label'].values, data_aug_train['term'].values):
  sent1_li.append(train_sent)
  sent2_li.append(label_sent)
  label_li.append(1.0)

  lvl0 = level0_dict[label]
  lvl1 = level1_dict[label]

  rnd_df = data_aug_train[(data_aug_train['term']!=term) & (data_aug_train['label']!=label)].sample(10, random_state = 42)
  for new_train_sent, new_label_sent, new_label, new_term in zip(rnd_df['term_definition'].values, rnd_df['label_definition'].values, rnd_df['label'].values, rnd_df['term'].values):
    lvl0_new = level0_dict[new_label]
    lvl1_new = level1_dict[new_label]
    if lvl1 == lvl1_new:
      sent1_li.append(train_sent)
      sent2_li.append(new_label_sent)
      label_li.append(2*k)
    elif lvl0 == lvl0_new:
      sent1_li.append(train_sent)
      sent2_li.append(new_label_sent)
      label_li.append(k)
    else:
      sent1_li.append(train_sent)
      sent2_li.append(new_label_sent)
      label_li.append(0.0)

data_pos_neg = pd.DataFrame({'term_or_def': sent1_li, 'label_or_def':sent2_li, 'sim':label_li})
data_pos_neg['sim'].value_counts()

0.0    12838
1.0     1440
0.8     1106
0.4      456
Name: sim, dtype: int64

# Undersampling to balance the dataset

In [ ]:
N = 1400
data_pos_neg_use = data_pos_neg[data_pos_neg['sim'] > 0].append(data_pos_neg[data_pos_neg['sim'] == 0].sample(N, random_state=42)).copy().fillna('')

# Model-1 using FinBERT

In [ ]:
model1 = SentenceTransformer('ProsusAI/finbert')
model_save_path = 'finbert_fin_hyp'
train_batch_size = 10 #20 #20 changed to 10 as CUDA out of memory
num_epochs = 25

distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE
margin = 0.5

train_samples_MultipleNegativesRankingLoss = []
train_samples_ConstrativeLoss = []

for sent1,sent2,sim in zip(data_pos_neg_use['term_or_def'].values, data_pos_neg_use['label_or_def'].values, data_pos_neg_use['sim'].values):
    train_samples_ConstrativeLoss.append(InputExample(texts=[sent1, sent2], label=sim))
    if sim == 1.0:
          train_samples_MultipleNegativesRankingLoss.append(InputExample(texts=[sent1, sent2], label=sim))
          train_samples_MultipleNegativesRankingLoss.append(InputExample(texts=[sent2, sent1], label=sim))  # if A is a duplicate of B, then B is a duplicate of A

# Create data loader and loss for MultipleNegativesRankingLoss
train_dataset_MultipleNegativesRankingLoss = SentencesDataset(train_samples_MultipleNegativesRankingLoss, model=model1)
train_dataloader_MultipleNegativesRankingLoss = DataLoader(train_dataset_MultipleNegativesRankingLoss, shuffle=True, batch_size=train_batch_size)
train_loss_MultipleNegativesRankingLoss = losses.MultipleNegativesRankingLoss(model1)


# Create data loader and loss for OnlineContrastiveLoss
train_dataset_ConstrativeLoss = SentencesDataset(train_samples_ConstrativeLoss, model=model1)
train_dataloader_ConstrativeLoss = DataLoader(train_dataset_ConstrativeLoss, shuffle=True, batch_size=train_batch_size)
train_loss_ConstrativeLoss = losses.OnlineContrastiveLoss(model=model1, distance_metric=distance_metric, margin=margin)

# .....
# Train the model
model1.fit(train_objectives=[(train_dataloader_MultipleNegativesRankingLoss, train_loss_MultipleNegativesRankingLoss), (train_dataloader_ConstrativeLoss, train_loss_ConstrativeLoss)],
          #evaluator=seq_evaluator,
          epochs=num_epochs,
          warmup_steps=1000,
          output_path=model_save_path
          )
model1.save('./'+model_save_path+"_save")

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/ProsusAI_finbert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/288 [00:00<?, ?it/s]

Iteration:   0%|          | 0/288 [00:00<?, ?it/s]

Iteration:   0%|          | 0/288 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/288 [00:00<?, ?it/s]

Iteration:   0%|          | 0/288 [00:00<?, ?it/s]

Iteration:   0%|          | 0/288 [00:00<?, ?it/s]

Iteration:   0%|          | 0/288 [00:00<?, ?it/s]

Iteration:   0%|          | 0/288 [00:00<?, ?it/s]

In [ ]:
from google.colab import files
!zip -r /content/finbert_fin_hyp_save.zip /content/finbert_fin_hyp_save
files.download('/content/finbert_fin_hyp_save.zip')

  adding: content/finbert_fin_hyp_save/ (stored 0%)
  adding: content/finbert_fin_hyp_save/special_tokens_map.json (deflated 40%)
  adding: content/finbert_fin_hyp_save/config.json (deflated 50%)
  adding: content/finbert_fin_hyp_save/config_sentence_transformers.json (deflated 26%)
  adding: content/finbert_fin_hyp_save/README.md (deflated 61%)
  adding: content/finbert_fin_hyp_save/tokenizer_config.json (deflated 48%)
  adding: content/finbert_fin_hyp_save/pytorch_model.bin (deflated 7%)
  adding: content/finbert_fin_hyp_save/1_Pooling/ (stored 0%)
  adding: content/finbert_fin_hyp_save/1_Pooling/config.json (deflated 47%)
  adding: content/finbert_fin_hyp_save/vocab.txt (deflated 53%)
  adding: content/finbert_fin_hyp_save/sentence_bert_config.json (deflated 4%)
  adding: content/finbert_fin_hyp_save/modules.json (deflated 53%)
  adding: content/finbert_fin_hyp_save/tokenizer.json (deflated 71%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import sentence_transformers
sentence_transformers.__version__

'2.2.0'

# Model-2 using FinISH

In [ ]:
!pip uninstall sentence_transformers --y
!pip install sentence_transformers==1.0.4

     |████████████████████████████████| 74 kB 2.4 MB/s 
     |████████████████████████████████| 3.8 MB 28.4 MB/s 
     |████████████████████████████████| 1.2 MB 37.4 MB/s 
     |████████████████████████████████| 6.5 MB 35.1 MB/s 
     |████████████████████████████████| 895 kB 39.2 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 41.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-py3-none-any.whl size=114306 sha256=7336d25fc292ccc67095fc5c6f3e2253126e654d5bc8b4b883bef4586dc683c4
  Stored in directory: /root/.cache/pip/wheels/36/29/9e/1d73f2751adb27f61c414d2656cef08c8353b9ad7cb7f149ef
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import sentence_transformers
sentence_transformers.__version__

'1.0.4'

In [ ]:
#!git clone https://huggingface.co/yseop/roberta-base-finance-hypernym-identification

Cloning into 'roberta-base-finance-hypernym-identification'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 48 (delta 19), reused 0 (delta 0)
Unpacking objects: 100% (48/48), done.


In [ ]:
model2 =  SentenceTransformer('yseop/roberta-base-finance-hypernym-identification')#SentenceTransformer("/content/roberta-base-finance-hypernym-identification")#
model_save_path = 'finish_fin_hyp'
train_batch_size = 10 #30 changed to 10 as CUDA out of memory
num_epochs = 20 #45 changed to 20 as timeput was happening

distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE
margin = 0.5

train_samples_MultipleNegativesRankingLoss = []
train_samples_ConstrativeLoss = []

for sent1,sent2,sim in zip(data_pos_neg_use['term_or_def'].values, data_pos_neg_use['label_or_def'].values, data_pos_neg_use['sim'].values):
    train_samples_ConstrativeLoss.append(InputExample(texts=[sent1, sent2], label=sim))
    if sim == 1.0:
        train_samples_MultipleNegativesRankingLoss.append(InputExample(texts=[sent1, sent2], label=sim))
        train_samples_MultipleNegativesRankingLoss.append(InputExample(texts=[sent2, sent1], label=sim))  # if A is a duplicate of B, then B is a duplicate of A

# Create data loader and loss for MultipleNegativesRankingLoss
train_dataset_MultipleNegativesRankingLoss = SentencesDataset(train_samples_MultipleNegativesRankingLoss, model=model2)
train_dataloader_MultipleNegativesRankingLoss = DataLoader(train_dataset_MultipleNegativesRankingLoss, shuffle=True, batch_size=train_batch_size)
train_loss_MultipleNegativesRankingLoss = losses.MultipleNegativesRankingLoss(model2)


# Create data loader and loss for OnlineContrastiveLoss
train_dataset_ConstrativeLoss = SentencesDataset(train_samples_ConstrativeLoss, model=model2)
train_dataloader_ConstrativeLoss = DataLoader(train_dataset_ConstrativeLoss, shuffle=True, batch_size=train_batch_size)
train_loss_ConstrativeLoss = losses.OnlineContrastiveLoss(model=model2, distance_metric=distance_metric, margin=margin)

# .....
# Train the model
model2.fit(train_objectives=[(train_dataloader_MultipleNegativesRankingLoss, train_loss_MultipleNegativesRankingLoss), (train_dataloader_ConstrativeLoss, train_loss_ConstrativeLoss)],
          #evaluator=seq_evaluator,
          epochs=num_epochs,
          warmup_steps=1000,
          output_path=model_save_path
          )
model2.save('./'+model_save_path+"_save")

from google.colab import files
!zip -r /content/finish_fin_hyp_save.zip /content/finish_fin_hyp_save
files.download('/content/finish_fin_hyp_save.zip')

Exception when trying to download http://sbert.net/models/yseop/roberta-base-finance-hypernym-identification.zip. Response 404
SentenceTransformer-Model http://sbert.net/models/yseop/roberta-base-finance-hypernym-identification.zip not found. Try to create it from scratch
Try to create Transformer Model yseop/roberta-base-finance-hypernym-identification with mean pooling


Downloading:   0%|          | 0.00/671 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/288 [00:00<?, ?it/s]

Iteration:   0%|          | 0/288 [00:00<?, ?it/s]

  adding: content/finish_fin_hyp_save/ (stored 0%)
  adding: content/finish_fin_hyp_save/modules.json (deflated 55%)
  adding: content/finish_fin_hyp_save/config.json (stored 0%)
  adding: content/finish_fin_hyp_save/1_Pooling/ (stored 0%)
  adding: content/finish_fin_hyp_save/1_Pooling/config.json (deflated 47%)
  adding: content/finish_fin_hyp_save/0_Transformer/ (stored 0%)
  adding: content/finish_fin_hyp_save/0_Transformer/sentence_bert_config.json (deflated 4%)
  adding: content/finish_fin_hyp_save/0_Transformer/special_tokens_map.json (deflated 50%)
  adding: content/finish_fin_hyp_save/0_Transformer/config.json (deflated 48%)
  adding: content/finish_fin_hyp_save/0_Transformer/tokenizer_config.json (deflated 76%)
  adding: content/finish_fin_hyp_save/0_Transformer/tokenizer.json (deflated 72%)
  adding: content/finish_fin_hyp_save/0_Transformer/pytorch_model.bin (deflated 7%)
  adding: content/finish_fin_hyp_save/0_Transformer/merges.txt (deflated 53%)
  adding: content/finish_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Evaluation

In [ ]:
corpus = list(label_df_new['label_definition'].values)

# Query sentences:
queries = list(data_aug_valid['term_definition'].values)

In [ ]:
def results_pred(model):

  corpus_embedding = model.enocde(corpus, convert_to_tensor=True)

  top_k = min(17, len(corpus))
  all_hits = []
  for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=17)
    all_hits.append(hits)

  lli = []
  for kk in all_hits:
    kk_trns = {}
    for i in kk[0]:
      kk_trns[i['corpus_id']] = i['score']
    lli.append([kk_trns[i] for i in range(17)])

  valid_y = list(data_aug_valid['label_text'].values)

  predicted_df = pd.DataFrame({'term' : list(data_aug_valid['term'].values), 'label':valid_y})
  predicted_df = pd.concat([predicted_df, pd.DataFrame(lli)], axis = 1)

  pddf = predicted_df.groupby(['term', 'label']).mean().reset_index()
  pddf['li'] = pddf[[i for i in range(17)]].values.tolist()

  pddf['pred_valid'] = [np.argmax(i) for i in pddf['li'].values]
  pddf['predictions_to_rank'] = [list(np.argsort(i)[::-1]) for  i in pddf['li'].values]
  print(compute_average_rank(list(pddf['label']),list(ppdf['predictions_to_rank'])), accuracy_score(list(pddf['label']), list(pddf['pred_valid'])))
  return pddf

# Ensemble

In [ ]:
results_v1 = results_pred(model1)
results_v2 = results(model2)

cols = ['term', 'label', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

ensembled_df = results_v1[cols].append(results_v2[cols]).reset_index()
ensembled_pddf = ensembled_df.groupby(['term', 'label']).mean().reset_index()
ensembled_pddf['li'] = ensembled_pddf[[i for i in range(17)]].values.tolist()

ensembled_pddf['pred_valid'] = [np.argmax(i) for i in ensembled_pddf['li'].values]
ensembled_pddf['predictions_to_rank'] = [list(np.argsort(i)[::-1]) for i in ensembled_pddf['li'].values]
print(compute_average_rank(list(ensembled_pddf['label']),list(ensembled_ppdf['predictions_to_rank'])), accuracy_score(list(ensembled_pddf['label']), list(ensembled_pddf['pred_valid'])))